# 15.3 项目实施

In [1]:
from selenium import webdriver
driver = webdriver.Firefox()
driver.get("https://www.dianping.com/search/category/7/10/p1")

WebDriverException: Message: Unable to find a matching set of capabilities


In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

caps = webdriver.DesiredCapabilities().FIREFOX
caps["marionette"] = False
binary = FirefoxBinary(r'D:\Program Files\Mozilla Firefox\firefox.exe')
#把上述地址改成你电脑中Firefox程序的地址
driver = webdriver.Firefox(firefox_binary=binary, capabilities=caps)
driver.get("https://www.dianping.com/search/category/7/10/p1")

In [3]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

caps = webdriver.DesiredCapabilities().FIREFOX
caps["marionette"] = False

binary = FirefoxBinary(r'D:\Program Files\Mozilla Firefox\firefox.exe')
#把上述地址改成你电脑中Firefox程序的地址
fp = webdriver.FirefoxProfile()
fp.set_preference("permissions.default.image",2)
fp.set_preference("javascript.enabled", False)

driver = webdriver.Firefox(firefox_binary=binary, firefox_profile = fp, capabilities=caps)
driver.get("https://www.dianping.com/search/category/7/10/p1")
driver.implicitly_wait(20)

In [2]:
from bs4 import BeautifulSoup
import csv
import time
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

caps = webdriver.DesiredCapabilities().FIREFOX
caps["marionette"] = False

binary = FirefoxBinary(r'D:\Program Files\Mozilla Firefox\firefox.exe')
#把上述地址改成你电脑中Firefox程序的地址
fp = webdriver.FirefoxProfile()
fp.set_preference("permissions.default.image",2)
fp.set_preference("javascript.enabled", False)

driver = webdriver.Firefox(firefox_binary=binary, firefox_profile = fp, capabilities=caps)
driver.implicitly_wait(20)

for i in range(1,51):
    link = "https://www.dianping.com/search/category/7/10/p" + str(i)
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, "lxml")
    
    output_list =[]
    output_list = outputOneCommentResult(i, soup, output_list)
        
    with open('restaurant_list.csv', 'a+', newline='', encoding='utf-8') as csvfile:
        spamwriter = csv.writer(csvfile,dialect='excel')
        spamwriter.writerows(output_list)
    time.sleep(2)

In [1]:
def outputOneCommentResult(page_id, soup, output_list):
    for item in soup.find(id='shop-all-list').ul:
        try:
            title = item.find(class_='tit').a.text.strip()
        except:
            title = ''
        try:
            link = "http://www.dianping.com" + item.find(class_='tit').a['href']
        except:
            link = ''
        try:
            star = item.find(class_='comment').span['title']
        except:
            star = ''
        try:
            comment_link = "http://www.dianping.com" + item.find(class_='review-num')['href']
        except:
            comment_link = ''
        try:
            comment = item.find(class_='review-num').b.text.strip()
        except:
            comment = ''
        try:
            price = item.find(class_='mean-price').b.text.strip()
        except:
            price = ''
        try:
            tag = item.find(class_='tag').text.strip()
        except:
            tag = ''
        try:
            addr = item.find(class_='addr').text.strip()
        except:
            addr = ''
        try:
            commentlist = item.find(class_='comment-list').text.strip()
            commentlist = commentlist.replace("\n","|")
        except:
            commentlist = ''

        if title != "":
            output_list.append([str(page_id), title, link, star ,comment_link, comment, price, tag, addr, commentlist])
    return output_list

## 15.3.2 获取餐厅详细信息

In [63]:
from bs4 import BeautifulSoup
import csv
import time
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

caps = webdriver.DesiredCapabilities().FIREFOX
caps["marionette"] = False

binary = FirefoxBinary(r'D:\Program Files\Mozilla Firefox\firefox.exe')
#把上述地址改成你电脑中Firefox程序的地址
fp = webdriver.FirefoxProfile()
fp.set_preference("permissions.default.image",2)
fp.set_preference("javascript.enabled", False)

driver = webdriver.Firefox(firefox_binary=binary, firefox_profile = fp, capabilities=caps)
driver.implicitly_wait(30)

with open('restaurant_list.csv', encoding = 'utf-8') as f:
    csv_file = csv.reader(f)
    link_list = [[row[1],row[2]] for row in csv_file]

for eachone in link_list:
    name = eachone[0]
    link = eachone[1]
    output_list = []
    driver.get(link)
    locator = (By.CLASS_NAME, 'content')
    WebDriverWait(driver, 20, 0.5).until(EC.presence_of_element_located(locator))
    
    soup = BeautifulSoup(driver.page_source, "lxml")
    
    dishes_list = []
    for eachone in soup.find(class_="recommend-name"):
        try:
            dishes = eachone['title'].strip() + eachone.em.text
            dishes_list.append(dishes)
        except:
            pass
        
    comment_list = []
    for eachone in soup.find(class_="content"):
        try:
            comment_tag = eachone.a.text
            comment_list.append(comment_tag)
        except:
            pass
        
    output_list.append([name, link,'|'.join(dishes_list), '|'.join(comment_list)])
    with open('restuarant_detail.csv', 'a+' , newline ='' ,encoding = 'utf-8') as csv_file:
        writer = csv.writer(csv_file, dialect = 'excel')
        writer.writerows(output_list)
    time.sleep(2)

PermissionError: [Errno 13] Permission denied: 'restuarant_detail.csv'